In [244]:
import numpy as np
import stellargraph as sg
import pandas as pd
from stellargraph import StellarGraph, StellarDiGraph
import networkx as nx
import networkx as nx
import pandas as pd
import numpy as np
import itertools
import os
import gc

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import stellargraph as sg
from stellargraph import globalvar
from stellargraph.mapper import GraphSAGENodeGenerator
from stellargraph.layer import GraphSAGE

from tensorflow.keras import layers, optimizers, losses, metrics, Model
from sklearn import preprocessing, feature_extraction, model_selection
from stellargraph import datasets
from IPython.display import display, HTML
import matplotlib.pyplot as plt

In [245]:
def l_patch_matrix(ini_array, rows_to_be_added):

    '''
    Khi thêm các nút mới vào ma trận, ta có thể làm như sau:
    Từ chối kết nối giữa các nút mới với nhau.
    Giữ nguyên các kết nối chéo bằng 1.
    '''
    
    if rows_to_be_added.shape[0] <= 1 or len(rows_to_be_added.shape) <= 1:
        diag = rows_to_be_added @ rows_to_be_added.T
        rows_to_be_added = rows_to_be_added / diag
    else:
        diag = np.array([i @ i.T for i in rows_to_be_added])
        rows_to_be_added = rows_to_be_added / diag[:, np.newaxis]
    

    arr = np.vstack((ini_array, rows_to_be_added))
    rows, cols = arr.shape
    pad_width = ((0, max(rows, cols) - rows), (0, max(rows, cols) - cols))
    arr = np.pad(arr, pad_width, mode='constant')
    np.fill_diagonal(arr, 1)
    return arr




def add_node_to_X(ini_array:pd.DataFrame, row_to_be_added):

    """
    Thêm đặc trưng của nút vào X

    Nhận vào 2 đối số:
    ini_array: X của đồ thị
    row_to_be_added: đặc trưng của nút mới
    """

    ini_array.loc[len(ini_array.index)] = row_to_be_added
    return ini_array

In [246]:
adj_matr = pd.read_csv('./input/m1.csv', index_col=None, header=None)
adj_matr_m2 = pd.read_csv('./input/m2.csv', index_col=None, header=None)
adj_matr_m3 = pd.read_csv('./input/m3.csv', index_col=None, header=None)
adj_matr_m4 = pd.read_csv('./input/m4.csv', index_col=None, header=None)

In [247]:
diminutive = np.genfromtxt('input/diminutive_m1.csv', delimiter=',')
diminutive_m2 = np.genfromtxt('input/diminutive_m2.csv', delimiter=',')
diminutive_m3 = np.genfromtxt('input/diminutive_m3.csv', delimiter=',')
diminutive_m4 = np.genfromtxt('input/diminutive_m4.csv', delimiter=',')

In [248]:
X = pd.read_csv('input/app_api.csv', header=0, index_col='Unnamed: 0')
X_row, X_col = X.shape
labels = X.pop('Label')

In [249]:
#this is strictly for m1
adj_matr = adj_matr.to_numpy()
adj_matr = adj_matr / np.amax(adj_matr, axis=1)
adj_matr = np.tril(adj_matr)

#this is strictly for m2
adj_matr_m2 = adj_matr_m2.to_numpy()
adj_matr_m2 = adj_matr_m2 / np.amax(adj_matr_m2, axis=1)
adj_matr_m2 = np.tril(adj_matr_m2)

#this is strictly for m3
adj_matr_m3 = adj_matr_m3.to_numpy()
adj_matr_m3 = adj_matr_m3 / np.amax(adj_matr_m3, axis=1)
adj_matr_m3 = np.tril(adj_matr_m3)

#this is strictly for m4
adj_matr_m4 = adj_matr_m4.to_numpy()
adj_matr_m4 = adj_matr_m4 / np.amax(adj_matr_m4, axis=1)
adj_matr_m4 = np.tril(adj_matr_m4)

In [250]:
X_test = pd.read_csv('input/app_api_test_1000.csv', header=0, index_col='Unnamed: 0') #X_test là tập dữ liệu chứa các đặc trưng của các nút mới
X_test = X_test[200:399]
test_labels = X_test.pop('Label')
test_row, test_col = X_test.shape

In [251]:
test_index = np.arange(X_row, X_row + test_row, step=1)

In [252]:
X_full = pd.concat([X, X_test], axis=0, ignore_index=True)

In [253]:
A1 = X_test.to_numpy() @ diminutive
A2 = X_test.to_numpy() @ diminutive_m2
A3 = X_test.to_numpy() @ diminutive_m3
A4 = X_test.to_numpy() @ diminutive_m4

In [254]:
adj_matr.shape

(5000, 5000)

In [255]:
adj_matr = l_patch_matrix(adj_matr, A1)
adj_matr_m2 = l_patch_matrix(adj_matr_m2, A2)
adj_matr_m3 = l_patch_matrix(adj_matr_m3, A3)
adj_matr_m4 = l_patch_matrix(adj_matr_m4, A4)

In [257]:
G = nx.Graph(adj_matr)
graph_full = sg.StellarGraph.from_networkx(G, node_features=X_full)
del G, adj_matr

G_m2 = nx.Graph(adj_matr_m2)
graph_full_m2 = sg.StellarGraph.from_networkx(G_m2, node_features=X_full)
del G_m2, adj_matr_m2

G_m3 = nx.Graph(adj_matr_m3)
graph_full_m3 = sg.StellarGraph.from_networkx(G_m3, node_features=X_full)
del G_m3, adj_matr_m3

G_m4 = nx.Graph(adj_matr_m4)
graph_full_m4 = sg.StellarGraph.from_networkx(G_m4, node_features=X_full)
del G_m4, adj_matr_m4

print(graph_full.info())

gc.collect()

StellarGraph: Undirected multigraph
 Nodes: 5199, Edges: 13437669

 Node types:
  default: [5199]
    Features: float32 vector, length 400
    Edge types: default-default->default

 Edge types:
    default-default->default: [13437669]
        Weights: range=[5.88301e-09, 1], mean=0.416206, std=0.329732
        Features: none


28

In [258]:
# import pickle
# pickle.dump(graph_full, open("./output/graph_full_m1.pkl", "wb"))
# pickle.dump(graph_full_m2, open("./output/graph_full_m2.pkl", "wb"))
# pickle.dump(graph_full_m3, open("./output/graph_full_m3.pkl", "wb"))
# pickle.dump(graph_full_m4, open("./output/graph_full_m4.pkl", "wb"))

In [259]:
# gf1= pickle.load(open("./output/graph_full_m1.pkl", "rb"))
# print(gf1.info())

In [260]:
test_index

array([5000, 5001, 5002, 5003, 5004, 5005, 5006, 5007, 5008, 5009, 5010,
       5011, 5012, 5013, 5014, 5015, 5016, 5017, 5018, 5019, 5020, 5021,
       5022, 5023, 5024, 5025, 5026, 5027, 5028, 5029, 5030, 5031, 5032,
       5033, 5034, 5035, 5036, 5037, 5038, 5039, 5040, 5041, 5042, 5043,
       5044, 5045, 5046, 5047, 5048, 5049, 5050, 5051, 5052, 5053, 5054,
       5055, 5056, 5057, 5058, 5059, 5060, 5061, 5062, 5063, 5064, 5065,
       5066, 5067, 5068, 5069, 5070, 5071, 5072, 5073, 5074, 5075, 5076,
       5077, 5078, 5079, 5080, 5081, 5082, 5083, 5084, 5085, 5086, 5087,
       5088, 5089, 5090, 5091, 5092, 5093, 5094, 5095, 5096, 5097, 5098,
       5099, 5100, 5101, 5102, 5103, 5104, 5105, 5106, 5107, 5108, 5109,
       5110, 5111, 5112, 5113, 5114, 5115, 5116, 5117, 5118, 5119, 5120,
       5121, 5122, 5123, 5124, 5125, 5126, 5127, 5128, 5129, 5130, 5131,
       5132, 5133, 5134, 5135, 5136, 5137, 5138, 5139, 5140, 5141, 5142,
       5143, 5144, 5145, 5146, 5147, 5148, 5149, 51

In [261]:
target_encoding = preprocessing.LabelBinarizer()

targets = target_encoding.fit_transform(labels)

In [262]:
batch_size = 100
num_samples = [10, 10]

generator = GraphSAGENodeGenerator(graph_full, batch_size, num_samples, weighted=True)
gen = generator.flow(test_index, test_labels)

generator_m2 = GraphSAGENodeGenerator(graph_full_m2, batch_size, num_samples, weighted=True)
gen_m2 = generator_m2.flow(test_index, test_labels)

generator_m3 = GraphSAGENodeGenerator(graph_full_m3, batch_size, num_samples, weighted=True)
gen_m3 = generator_m3.flow(test_index, test_labels)

generator_m4 = GraphSAGENodeGenerator(graph_full_m4, batch_size, num_samples, weighted=True)
gen_m4 = generator_m4.flow(test_index, test_labels)

In [263]:
input_1 = np.vstack(np.array([gen[i][0][0] for i in range(len(gen))]))
input_2 = np.vstack(np.array([gen[i][0][1] for i in range(len(gen))]))
input_3 = np.vstack(np.array([gen[i][0][2] for i in range(len(gen))]))

# label = np.vstack([gen[i][1] for i in range(len(gen))])

del gen
gc.collect()

input_4 = np.vstack(np.array([gen_m2[i][0][0] for i in range(len(gen_m2))]))
input_5 = np.vstack(np.array([gen_m2[i][0][1] for i in range(len(gen_m2))]))
input_6 = np.vstack(np.array([gen_m2[i][0][2] for i in range(len(gen_m2))]))

del gen_m2
gc.collect()

input_7 = np.vstack(np.array([gen_m3[i][0][0] for i in range(len(gen_m3))]))
input_8 = np.vstack(np.array([gen_m3[i][0][1] for i in range(len(gen_m3))]))
input_9 = np.vstack(np.array([gen_m3[i][0][2] for i in range(len(gen_m3))]))

del gen_m3
gc.collect()

input_10 = np.vstack(np.array([gen_m4[i][0][0] for i in range(len(gen_m4))]))
input_11 = np.vstack(np.array([gen_m4[i][0][1] for i in range(len(gen_m4))]))
input_12 = np.vstack(np.array([gen_m4[i][0][2] for i in range(len(gen_m4))]))

del gen_m4
gc.collect()

/root/miniconda3/envs/nckh/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.
/root/miniconda3/envs/nckh/lib/python3.7/site-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  
/root/miniconda3/envs/nckh/lib/python3.7/site-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is depre

18

In [264]:
graphsage_model = GraphSAGE(layer_sizes=[64, 32], generator=generator, bias=True, dropout=0.5,)

graphsage_model_m2 = GraphSAGE(layer_sizes=[64, 32], generator=generator_m2, bias=True, dropout=0.5,)

graphsage_model_m3 = GraphSAGE(layer_sizes=[64, 32], generator=generator_m3, bias=True, dropout=0.5,)

graphsage_model_m4 = GraphSAGE(layer_sizes=[64, 32], generator=generator_m4, bias=True, dropout=0.5,)

x_inp, x_out = graphsage_model.in_out_tensors()
prediction = layers.Dense(units=targets.shape[1], activation="softmax")(x_out)

x_inp_m2, x_out_m2 = graphsage_model_m2.in_out_tensors()
prediction_m2 = layers.Dense(units=targets.shape[1], activation="softmax")(x_out_m2)

x_inp_m3, x_out_m3 = graphsage_model_m3.in_out_tensors()
prediction_m3 = layers.Dense(units=targets.shape[1], activation="softmax")(x_out_m3)

x_inp_m4, x_out_m4 = graphsage_model_m4.in_out_tensors()
prediction_m4 = layers.Dense(units=targets.shape[1], activation="softmax")(x_out_m4)

model = Model(inputs=x_inp, outputs=prediction)
model.compile(optimizer=optimizers.Adam(learning_rate=0.005), loss=losses.categorical_crossentropy, metrics=["acc"],)

model_m2 = Model(inputs=x_inp_m2, outputs=prediction_m2)
model_m2.compile( optimizer=optimizers.Adam(learning_rate=0.005), loss=losses.categorical_crossentropy, metrics=["acc"],)

model_m3 = Model(inputs=x_inp_m3, outputs=prediction_m3)
model_m3.compile( optimizer=optimizers.Adam(learning_rate=0.005), loss=losses.categorical_crossentropy, metrics=["acc"],)

model_m4 = Model(inputs=x_inp_m4, outputs=prediction_m4)
model_m4.compile(optimizer=optimizers.Adam(learning_rate=0.005),loss=losses.categorical_crossentropy,metrics=["acc"],)

combined = layers.concatenate([model.output, model_m2.output, model_m3.output, model_m4.output])

z = layers.Dense(units=targets.shape[1], activation="softmax")(combined)

combined_model = Model(inputs=[x_inp, x_inp_m2, x_inp_m3, x_inp_m4], outputs=z)


combined_model.compile(

optimizer=optimizers.Adam(learning_rate=0.005),

loss=losses.categorical_crossentropy,

metrics=["acc"],

)

combined_model.load_weights('./input/model.h5')

/root/miniconda3/envs/nckh/lib/python3.7/site-packages/keras/initializers/initializers_v2.py:121: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  f"The initializer {self.__class__.__name__} is unseeded "


In [265]:
targets.shape[1]

5

In [266]:
hold_out_predictions = combined_model.predict([input_1, input_2, input_3, input_4, input_5, input_6, input_7, input_8, input_9, input_10, input_11, input_12])

7/7 [==============================] - 1s 5ms/step


In [267]:
hold_out_predictions = target_encoding.inverse_transform(hold_out_predictions)

In [268]:
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score

a = pd.DataFrame()
a.insert(0, "1", test_labels)
a.insert(1, "2", hold_out_predictions)
a.to_csv("output/predict_result.csv")

print(accuracy_score(test_labels, hold_out_predictions))
# print(recall_score(test_labels, hold_out_predictions, average='macro'))
# print(f1_score(test_labels, hold_out_predictions, average='macro'))
# print(precision_score(test_labels, hold_out_predictions, average='macro'))



0.2864321608040201


In [269]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_labels, hold_out_predictions)

array([[  1,   0,   0,   0,   0],
       [  3,  56,   3,  21, 115],
       [  0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0]])